# Data Cleaning with PySpark

In [1]:
# Here we import all the tools from spark that we will use
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, concat_ws, lit, regexp_replace, lower, to_timestamp, hour, count
from pyspark.sql.types import IntegerType

In [2]:
# Now we start new spark session, so that we can work with big datasets easily
spark = SparkSession.builder.appName("DataCleaning1").getOrCreate()

25/04/13 22:53:53 WARN Utils: Your hostname, Huis-Surface-Laptop-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.211 instead (on interface en0)
25/04/13 22:53:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 22:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Here we read the CSV file into spark with headers and types are done automatically
df = spark.read.csv('Motor_Vehicle_Collisions_-_Crashes.csv', header=True, inferSchema=True)

In [4]:
# Here we check the structure and take a look at first 5 entries
df.printSchema()
df.show(5)

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: string (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: string (nullable = true)
 |-- CONTRIBUTING FAC

25/04/13 22:54:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+----------+--------+--------+--------+----------+--------------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME| BOROUGH|ZIP CODE|LATITUDE| LONGITUDE|      ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING FA

## 1. Filling Missing Values

In [5]:
# 1 A. Filling in the empty values with 'Unspecified' for "CONTRIBUTING FACTOR VEHICLE 1,2,3,4,5"

# Initially we replace the values with nan and then fill it with 'Unspecified'
contributing_factor_columns = [col for col in df.columns if 'CONTRIBUTING FACTOR VEHICLE' in col]
for column in contributing_factor_columns:
    df = df.withColumn(column, when(col(column).isin(['nan', 'NaN', 'None', '', ' ', 'N/A', '1', '80']), None).otherwise(col(column)))
    df = df.fillna({column: 'Unspecified'})

In [6]:
# 1B. Here we replace them missing number with 0 in all the columns with name as 'NUMBER' as their name
number_columns = [col for col in df.columns if 'NUMBER' in col]
for column in number_columns:
    df = df.fillna({column: 0})

In [7]:
# 1C. Replacing 0 values with None in 'LATITUDE', 'LONGITUDE', and 'ZIP CODE'
df = df.withColumn('LATITUDE', when(col('LATITUDE') == 0, None).otherwise(col('LATITUDE')))
df = df.withColumn('LONGITUDE', when(col('LONGITUDE') == 0, None).otherwise(col('LONGITUDE')))
df = df.withColumn('ZIP CODE', when(col('ZIP CODE') == 0, None).otherwise(col('ZIP CODE')))

## 2. Shifting Attributes

In [8]:
# 2A. Filling empty vehicle types by copying the next columns value if its there
vehicle_columns = [col for col in df.columns if 'VEHICLE TYPE CODE' in col]
for i in range(len(vehicle_columns) - 1):
    df = df.withColumn(vehicle_columns[i], when(col(vehicle_columns[i]).isNull(), col(vehicle_columns[i + 1])).otherwise(col(vehicle_columns[i])))

## 3. Changing Case

In [9]:
# 3A. Here we comine all the street names into one columns 'Addresses' with no extra spaces.
df = df.withColumn('Addresses', concat_ws(' ', lower(col('ON STREET NAME')), lower(col('CROSS STREET NAME')), lower(col('OFF STREET NAME'))))
df = df.withColumn('Addresses', regexp_replace(col('Addresses'), '\s+', ' '))

In [10]:
# 3B. Here we are turning all the behicle type values in lower case 
for column in vehicle_columns:
    df = df.withColumn(column, lower(col(column)))

In [11]:
# 3C. Here we are making all the contributing factor values to lower case
for column in contributing_factor_columns:
    df = df.withColumn(column, lower(col(column)))

## 4. Fixing Date and Time

In [12]:
# Now we combine the crash date and crash time to a single timestamp that is consistent
df = df.withColumn('CRASH DATE & TIME', to_timestamp(concat_ws(' ', col('CRASH DATE'), col('CRASH TIME'))))

## 5. Removing Duplicate Values

In [13]:
# Now we remove all the duplicate rows that have the same crash time and location
df = df.dropDuplicates(['CRASH DATE & TIME', 'LATITUDE', 'LONGITUDE'])

## 6. Creating Time of Day Attribute

In [14]:
# Here we label every crash by time of the day based on the hour of the crash took place
df = df.withColumn('Time of Day', when((hour(col('CRASH DATE & TIME')) >= 5) & (hour(col('CRASH DATE & TIME')) < 12), 'morning')
                           .when((hour(col('CRASH DATE & TIME')) >= 12) & (hour(col('CRASH DATE & TIME')) < 17), 'afternoon')
                           .when((hour(col('CRASH DATE & TIME')) >= 17) & (hour(col('CRASH DATE & TIME')) < 21), 'evening')
                           .otherwise('night'))

## 7. Changing Data Types

In [15]:
# Now we convert zip code, injuries and deaths to integers 
df = df.withColumn('ZIP CODE', col('ZIP CODE').cast(IntegerType()))
df = df.withColumn('NUMBER OF PERSONS INJURED', col('NUMBER OF PERSONS INJURED').cast(IntegerType()))
df = df.withColumn('NUMBER OF PERSONS KILLED', col('NUMBER OF PERSONS KILLED').cast(IntegerType()))

## 8. Dropping Unnecessary Columns

In [16]:
# Dropping all the extra columns, since we already have this information in other column
columns_to_drop = ['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'CRASH DATE', 'CRASH TIME']
df = df.drop(*columns_to_drop)

In [17]:
# Now finally we move the crash date and time to the front so as to make the table easier to read.
columns = ['CRASH DATE & TIME'] + [col for col in df.columns if col != 'CRASH DATE & TIME']
df = df.select(*columns)

## Saving the Cleaned Data

In [18]:
# Now since all the processing is done we save the output to new CSV folder called "Intermediate"
df.write.mode("overwrite").option("header", True).csv("Intermediate")

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=30453Kb max_used=30747Kb free=100618Kb
 bounds [0x00000001069e0000, 0x0000000108820000, 0x000000010e9e0000]
 total_blobs=10531 nmethods=9603 adapters=840
 compilation: disabled (not enough contiguous free space left)
